<a href="https://colab.research.google.com/github/ageofthegit/trading-local/blob/main/GCT02_0_BT_w_bolband.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
INSTALL_PACKAGES = True

In [3]:
EXPORT = True

In [4]:
!pwd

/content


In [5]:

if INSTALL_PACKAGES:
  !pip install backtrader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.1 MB/s eta 0:00:00


In [6]:
import backtrader as bt

In [7]:
INSTALL_PACKAGES = False

In [ ]:
#### IMPORTING THE RELEVANT FILES

In [8]:

# Importing Library

import pandas as pd, numpy as np, csv
import datetime, pytz, inspect
# import vectorbt as vbt

local_tz = pytz.timezone('Asia/Kolkata')  # Replace with your time zone
#from sktime import sliding_window_view


In [9]:

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


In [10]:

#file_path = "/content/drive/Computers/Omen/Gdrive/data-quant/BTC"
file_path = "/content/drive/MyDrive/Quantvaala/data"
model_path = "/content/drive/MyDrive/Quantvaala/model"

logs_path = "/content/drive/MyDrive/Quantvaala/logs"
output_path = "/content/drive/MyDrive/Quantvaala/output"

symbol = "BTC"
model_path = model_path + '/' + symbol
file_path = file_path + '/' + symbol

ctime = "1h"

if ctime == "1m":
  freq_pft = "1T"
if ctime == "3m":
  freq_pft = "3T"
if ctime == "5m":
  freq_pft = "5T"
if ctime == "15m":
  freq_pft = "15T"
if ctime == "30m":
  freq_pft = "30T"
if ctime == "1h":
  freq_pft = "1H"
if ctime == "2h":
  freq_pft = "2H"
if ctime == "4h":
  freq_pft = "4H"
if ctime == "6h":
  freq_pft = "6H"
# if ctime == "1d":
#   freq_pft = "d"



vars_basc = {symbol : symbol, ctime : ctime}
vars_back = {}
vars_strt = {}
vars_adnl = {}

file1 = f"data_2020_2023_{ctime}.csv"
# file2 = "data_2020_2023_15min.csv"

pricesdf1 = pd.read_csv(f"{file_path}/{file1}", index_col=0)
print(pricesdf1.columns.values.tolist())
print(pricesdf1.shape)

# pricesdf2 = pd.read_csv(f"{file_path}/{file2}", index_col=0)
# print(pricesdf2.columns.values.tolist())
# print(pricesdf2.shape)


# print(f"Importing {file1} for {symbol}")
# pricesdf = pd.read_csv(f"{file_path}", index_col=0)

# Use pd.concat to combine them
print()
#pricesdf = pd.concat( [pricesdf1, pricesdf2], ignore_index=True )

# pricesdf = pd.concat( [pricesdf1, pricesdf2])
pricesdfraw = pricesdf1.copy()
# pricesdf[['open','high','low','close']] = pricesdf[['open','high','low','close']]#.astype('float64')
# pricesdf[['open','high','low','close']] = pricesdf[['open','high','low','close']] / 100000


print(pricesdfraw.columns.values.tolist())
print(pricesdfraw.shape)

del pricesdf1


['open', 'high', 'low', 'close', 'volume']
(35033, 5)

['open', 'high', 'low', 'close', 'volume']
(35033, 5)


In [11]:

display(pricesdfraw.head())
display(pricesdfraw.tail())


,open,high,low,close,volume
datetime,,,,,
2020-01-01 05:30:00,7195.24,7196.25,7175.46,7177.02,511.814901
2020-01-01 06:30:00,7176.47,7230.00,7175.71,7216.27,883.052603
2020-01-01 07:30:00,7215.52,7244.87,7211.41,7242.85,655.156809
2020-01-01 08:30:00,7242.66,7245.00,7220.00,7225.01,783.724867
2020-01-01 09:30:00,7225.00,7230.00,7215.03,7217.27,467.812578


,open,high,low,close,volume
datetime,,,,,
2024-01-01 01:30:00,42619.04,42680.36,42504.56,42558.01,1015.14035
2024-01-01 02:30:00,42558.02,42670.44,42520.72,42520.72,923.43813
2024-01-01 03:30:00,42520.73,42591.10,42056.00,42257.88,1811.59411
2024-01-01 04:30:00,42257.89,42345.65,42066.00,42283.58,1710.13721
2024-01-01 05:30:00,42283.58,42554.57,42261.02,42475.23,1271.68108


In [12]:

# Generating Train x Test Split at 80% and 20%

pricesdfraw.index = pd.to_datetime(pricesdfraw.index, format='%Y-%m-%d %H:%M:%S')

# columns_for_prediction = ["close","volume"]

# data_train = pd.DataFrame(pricesdf[columns_for_prediction][0: int(len(pricesdf)*0.80)])
# print(data_train.shape)
# display(data_train.tail(5))

# data_test = pd.DataFrame(pricesdf[columns_for_prediction][int(len(pricesdf)*0.80): len(pricesdf)])
# print(data_test.shape)
# display(data_test.tail(5))


In [13]:

start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-12-31')

print(pricesdfraw.shape)
pricesdf = pricesdfraw[ (pricesdfraw.index >= start_date) & (pricesdfraw.index <= end_date) ]
# pricesdf.index = pd.to_datetime(pricesdf.index, format='%Y-%m-%d %H:%M:%S')
print(pricesdf.shape)


(35033, 5)
(35003, 5)


In [14]:

print(pricesdf.columns.values.tolist())
print()

display(pricesdf.head(25))


['open', 'high', 'low', 'close', 'volume']



,open,high,low,close,volume
datetime,,,,,
2020-01-01 05:30:00,7195.24,7196.25,7175.46,7177.02,511.814901
2020-01-01 06:30:00,7176.47,7230.00,7175.71,7216.27,883.052603
2020-01-01 07:30:00,7215.52,7244.87,7211.41,7242.85,655.156809
2020-01-01 08:30:00,7242.66,7245.00,7220.00,7225.01,783.724867
2020-01-01 09:30:00,7225.00,7230.00,7215.03,7217.27,467.812578
2020-01-01 10:30:00,7217.26,7229.76,7216.65,7224.21,344.670596
2020-01-01 11:30:00,7224.24,7236.27,7221.51,7225.62,621.467023
2020-01-01 12:30:00,7225.88,7232.94,7199.11,7209.83,627.344854
2020-01-01 13:30:00,7209.83,7210.00,7180.00,7200.64,915.545974


In [121]:

close_prices = pricesdf['close']
print(close_prices.shape)


(35003,)


In [122]:
## DEFINING THE STRATEGY IN PART(s)

In [247]:

if 'BBands' in globals():
  del BBands

class BBands(bt.Strategy):
    # params = (('period', 20), ('devfactor', 2), ('direction','long'))
    params = dict(period=12, devfactor=2, dir='long', trailpct = 0.01)

    def __init__(self):

      self.dataclose = self.data.close
      self.dataopen = self.data.open
      self.datahigh = self.data.high
      self.datalow = self.data.low

      self.bbands = bt.indicators.BollingerBands(self.dataclose, period=self.params.period, devfactor=self.params.devfactor)
      self.cross_up = bt.indicators.CrossUp(self.dataclose, self.bbands.lines.top)
      self.cross_dwn = bt.indicators.CrossDown(self.dataclose, self.bbands.lines.bot)

      self.trade_log = open(f'{logs_path}/trade_log.csv', 'a', newline='')
      self.csv_writer = csv.writer(self.trade_log)
      self.csv_writer.writerow(['Timestamp', 'Entry', 'Exit', 'Size', 'PnL', 'Direction', 'Status'])

      self.bb_data = {'date': [], 'open': [] , 'high': [] , 'low': [], 'cclose': []
                      , 'bbtop': []
                      , 'high_since_entry' : [], 'trail_stop_price' : []
                      , 'cross_up': [], 'cross_dwn': []
                      , 'cash': [], 'buyz': [], 'sellz': [], 'selltype':[]
                      # , 'in_position': []
                      , 'pos_size': [], 'pnl':[],  'commission' : []}

      self.high_since_entry = None
      self.trail_stop_price = None
      self.order = None
      self.current_row = 0


In [248]:

# Print the source code of the extended __init__ method
# print("Extended __init__ method:")
print(inspect.getsource(BBands.__init__))


    def __init__(self):

      self.dataclose = self.data.close
      self.dataopen = self.data.open
      self.datahigh = self.data.high
      self.datalow = self.data.low

      self.bbands = bt.indicators.BollingerBands(self.dataclose, period=self.params.period, devfactor=self.params.devfactor)
      self.cross_up = bt.indicators.CrossUp(self.dataclose, self.bbands.lines.top)
      self.cross_dwn = bt.indicators.CrossDown(self.dataclose, self.bbands.lines.bot)

      self.trade_log = open(f'{logs_path}/trade_log.csv', 'a', newline='')
      self.csv_writer = csv.writer(self.trade_log)
      self.csv_writer.writerow(['Timestamp', 'Entry', 'Exit', 'Size', 'PnL', 'Direction', 'Status'])

      self.bb_data = {'date': [], 'open': [] , 'high': [] , 'low': [], 'cclose': []
                      , 'bbtop': []
                      , 'high_since_entry' : [], 'trail_stop_price' : [] 
                      , 'cross_up': [], 'cross_dwn': []
                      , 'cash': [], 'buyz': [], 'sell

In [249]:

print(inspect.getsource(BBands.next))


    def next(self):
        '''
        This method will be called for all remaining data points when the
        minimum period for all datas/indicators have been meet.
        '''
        pass



In [250]:

def next_(self):
  dir = self.params.dir
  cash = np.round(self.broker.getcash(),2)
  currprice =  self.dataclose[0]

  self.bb_data['date'].append(self.datas[0].datetime.datetime(0))

  self.bb_data['open'].append(self.dataopen[0])
  self.bb_data['high'].append(self.datahigh[0])
  self.bb_data['low'].append(self.datalow[0])
  self.bb_data['cclose'].append(self.dataclose[0])
  self.bb_data['bbtop'].append(self.bbands.lines.top[0])

  self.bb_data['cross_up'].append(self.cross_up[0])
  self.bb_data['cross_dwn'].append(self.cross_dwn[0])
  self.bb_data['cash'].append(cash)
  # self.bb_data['in_position'].append(self.position)
  self.bb_data['pos_size'].append(self.position.size)

  buyz = False
  sellz = False
  selltype = '-'

  if dir == 'long':

    if self.position.size == 0:
      if self.cross_up:
        # self.buy(size=1, exectype=bt.Order.StopTrail, trailpercent=self.params.trailpct)
        self.buy(size=1)
        self.sell(size=1, exectype=bt.Order.StopTrail, trailpercent=self.params.trailpct)
        self.log(f'\t\tBUY {currprice:.2f}, Cash : {cash}, Close : {self.dataclose[0]:.2f}')
        buyz = True
        self.high_since_entry = currprice
        self.trail_stop_price = currprice*( 1 - self.params.trailpct)

    elif self.position.size > 0:
      if currprice > self.high_since_entry:
        self.high_since_entry = currprice
        self.trail_stop_price = self.high_since_entry*( 1 - self.params.trailpct )

      if self.cross_dwn:
          self.sell(size=self.position.size)
          self.log(f'\t\tSELL {currprice:.2f}, Cash : {cash:.2f}, Close : {self.dataclose[0]:.2f}')
          sellz = True
          selltype = 'SELL'

  self.bb_data['buyz'].append(buyz)
  self.bb_data['sellz'].append(sellz)
  self.bb_data['selltype'].append(selltype)

  # Appending Dummy Values, as most changing will be handled in the Notify-Trade
  self.bb_data['pnl'].append(0)
  self.bb_data['commission'].append(0)

  # Appending trailing stgop variable(s)
  self.bb_data['high_since_entry'].append(self.high_since_entry)
  self.bb_data['trail_stop_price'].append(self.trail_stop_price)

  self.current_row += 1

BBands.next = next_

print(inspect.getsource(BBands.next))


def next_(self):
  dir = self.params.dir
  cash = np.round(self.broker.getcash(),2)
  currprice =  self.dataclose[0]

  self.bb_data['date'].append(self.datas[0].datetime.datetime(0))
  
  self.bb_data['open'].append(self.dataopen[0])
  self.bb_data['high'].append(self.datahigh[0])
  self.bb_data['low'].append(self.datalow[0])
  self.bb_data['cclose'].append(self.dataclose[0])
  self.bb_data['bbtop'].append(self.bbands.lines.top[0])
    
  self.bb_data['cross_up'].append(self.cross_up[0])
  self.bb_data['cross_dwn'].append(self.cross_dwn[0])
  self.bb_data['cash'].append(cash)
  # self.bb_data['in_position'].append(self.position)
  self.bb_data['pos_size'].append(self.position.size)

  buyz = False
  sellz = False
  selltype = '-'

  if dir == 'long':

    if self.position.size == 0:
      if self.cross_up:
        # self.buy(size=1, exectype=bt.Order.StopTrail, trailpercent=self.params.trailpct)
        self.buy(size=1)
        self.sell(size=1, exectype=bt.Order.StopTrail, trailperce

In [251]:

  # if dir == 'long':
  #   if self.position.size == 0:
  #       if self.cross_up:
  #         # self.buy(size=1)
  #         # self.order = self.buy(size=1)
  #         self.order = self.buy(size=1, exectype=bt.Order.StopTrail, trailpercent=self.params.trailpct)
  #         self.log(f'BUY {currprice:.2f}, Cash : {cash}, Close : {self.dataclose[0]:.2f}')
  #         self.high_since_entry = currprice
  #         self.trail_stop_price = currprice*( 1 - self.params.trailpct)

  #         # buyz = self.cross_up[0]
  #         buyz = True

  #   elif self.position.size > 0:
  #     # Update the highest price since entry, and hence the trail-stop
  #     if currprice > self.high_since_entry:
  #       self.high_since_entry = currprice
  #       self.trail_stop_price = self.high_since_entry*(1 - self.params.trailpct)

  #     # Check for cross-down or trailing-stop
  #     if self.cross_dwn:
  #         # self.order = self.sell(size=self.position.size)
  #         self.order = self.sell(size=1)

  #         self.log(f'SELL {currprice:.2f}, Cash : {cash:.2f}, Close : {self.dataclose[0]:.2f}')
  #         # sellz = self.cross_dwn[0]
  #         sellz = True
  #         selltype = 'SELL'

  #     elif currprice <= self.trail_stop_price:
  #         # self.order = self.sell(size=self.position.size)
  #         self.order = self.sell(size=1)
  #         # Sell with a trailing stop order
  #         # self.order = self.sell(size = abs(self.position.size), exectype=bt.Order.StopTrail, trailpercent=self.params.trailpct)
  #         sell_price = self.order.created.price
  #         self.log(f'CCPRICE {currprice:.2f}, Max Price : {self.high_since_entry}, Exp SP : {self.trail_stop_price}, SP : {sell_price} , Cash : {cash:.2f}, Close : {self.dataclose[0]:.2f}')
  #         sellz = True
  #         selltype = f'TRAIL STOP SELL @{self.params.trailpct}'

  # if dir =='both':
  #   if self.position.size == 0:
  #       if self.cross_up:
  #           self.buy(size=1)
  #           buyz = self.cross_up[0]
  #           self.log(f'BUY {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')

  #       elif self.cross_dwn:
  #           self.sell(size=1)
  #           sellz = self.cross_dwn[0]
  #           self.log(f'SHORT {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')

  #   elif self.position.size > 0:
  #       if self.cross_dwn:
  #           self.sell(size=self.position.size)  # Close long position
  #           self.log(f'SELL {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')
  #           self.sell(size=1)  # Open short position
  #           self.log(f'SHORT {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')
  #           sellz = self.cross_dwn[0]

  #   elif self.position.size < 0:
  #       if self.cross_up:
  #           self.close(size=abs(self.position.size))  # Close short position
  #           self.log(f'CLOSE SHORT {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')
  #           self.buy(size=1)  # Open long position
  #           self.log(f'BUY {self.dataclose[0]:.2f}, Cash : {self.broker.getcash():.2f}')
  #           buyz = self.cross_up[0]




In [252]:

def log_(self, txt, dt=None):
    dt = dt or self.datas[0].datetime.datetime(0)
    cash_balance = self.broker.getcash()
    # print(f'{dt.isoformat()} {txt}')
    formatted_dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    print(f'{formatted_dt} {txt}')
    row = {
        'date': formatted_dt,
        'bbtop': np.round(self.bb_data['bbtop'][-1], 2) if self.bb_data['bbtop'] else None,
        'cclose': self.dataclose[0],
        'cross_up': self.bb_data['cross_up'][-1] if self.bb_data['cross_up'] else None,
        'cross_dwn': self.bb_data['cross_dwn'][-1] if self.bb_data['cross_dwn'] else None,
        'cash': np.round(cash_balance, 2),
        'buyz': self.bb_data['buyz'][-1] if self.bb_data['buyz'] else None,
        'sellz': self.bb_data['sellz'][-1] if self.bb_data['sellz'] else None,
        'pos_size': self.bb_data['pos_size'][-1] if self.bb_data['pos_size'] else None
    }
    # print(row)

# Bind the extended log method to the class
BBands.log = log_

# print(inspect.getsource(BBands.log))


In [253]:

def notify_order(self, order):
    status = order.getstatusname()

    # if order.status in [order.Completed, order.Submitted, order.Accepted, order.Created]:
    if order.status in [order.Completed]:
        if order.isbuy():
            self.log(f'BUY {status} : Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, ExecType / StopTrail: {order.exectype} / {bt.Order.StopTrail}, Position Size: {self.position.size}')
        elif order.issell() and order.exectype == bt.Order.StopTrail:
            self.log(f'TRAIL SELL {status} : Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, ExecType / StopTrail: {order.exectype} / {bt.Order.StopTrail}, Position Size: {self.position.size}')
        elif order.issell():
            self.log(f'SELL {status} : Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, ExecType / StopTrail: {order.exectype} / {bt.Order.StopTrail}, Position Size: {self.position.size}')
    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        self.log(f'ORDER {status}')

BBands.notify_order = notify_order


In [254]:

def notify_trade_(self, trade):
  if trade.isclosed:
    self.log(f'\tTRADE: Trade closed. Gross PnL: {trade.pnl:.2f}, Net PnL: {trade.pnlcomm:.2f}')
    pnl = trade.pnl
    commission = trade.commission

    # self.bb_data['pnl'].append(pnl)
    # self.bb_data['commission'].append(commission)

    self.bb_data['pnl'][self.current_row - 1] = pnl
    self.bb_data['commission'][self.current_row - 1] = commission
  else:
    # self.bb_data['pnl'].append(0)
    # self.bb_data['commission'].append(0)

    self.bb_data['pnl'][self.current_row - 1] = 0
    self.bb_data['commission'][self.current_row - 1] = 0

  # trade_details = [
  #     datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
  #     trade.price,
  #     trade.price if trade.isclosed else '-',
  #     trade.size,
  #     trade.pnl if trade.isclosed else '-',
  #     'Long' if trade.long else 'Short',
  #     'Closed' if trade.isclosed else 'Open'
  #     ]
  #   # print(trade_details)
  #   self.csv_writer.writerow(trade_details)

# Bind the extended notify_trade method to the class
BBands.notify_trade = notify_trade_

# print(inspect.getsource(BBands.notify_trade))


In [232]:

# Print all attributes and methods of the class
# print(dir(BBands))

# Print only methods of the class
# methods = inspect.getmembers(BBands, predicate=inspect.isfunction)
# for method in methods:
#     print(method[0])


In [255]:

from backtrader.analyzers import TradeAnalyzer, SQN


In [256]:

if 'cerebroz' in locals():
  del cerebroz

# Initialize cerebro
cerebroz = bt.Cerebro()

# Set initial cash to 100,000
cashinit = int("100,000".replace(",", ""))
cerebroz.broker.set_cash(cashinit)

# Create a data feed
print(pricesdf.shape)
data = bt.feeds.PandasData(dataname=pricesdf)

# Add data feed to cerebro (assume 'data' is already defined and added)
cerebroz.adddata(data)

# Add the strategy
# cerebroz.addstrategy(BBands)
cerebroz.addstrategy(BBands, dir='long')

# Set the commission
cerebroz.broker.setcommission(commission=0.0015)

# Add analyzers
cerebroz.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebroz.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebroz.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Custom analyzers

# cerebroz.addanalyzer(FinalTrades, _name='final_trades')

# Print the starting cash
print('Starting Portfolio Value: %.2f' % cerebroz.broker.getvalue())


(35003, 5)
Starting Portfolio Value: 100000.00


In [ ]:

if 'results' in locals():
  print('Deleting result(s)')
  del results

# Run the backtest
results = cerebroz.run()


2020-01-03 09:30:00 		BUY 7210.19, Cash : 100000.0, Close : 7210.19
2020-01-03 10:30:00 BUY Completed : Price: 7210.20, Cost: 7210.20, Comm: 10.82, ExecType / StopTrail: 0 / 5, Position Size: 1
2020-01-03 19:30:00 TRAIL SELL Completed : Price: 7275.38, Cost: 7210.20, Comm: 10.91, ExecType / StopTrail: 5 / 5, Position Size: 0
2020-01-03 19:30:00 	TRADE: Trade closed. Gross PnL: 65.18, Net PnL: 43.45
2020-01-05 06:30:00 		BUY 7459.18, Cash : 100043.45, Close : 7459.18
2020-01-05 07:30:00 BUY Completed : Price: 7459.91, Cost: 7459.91, Comm: 11.19, ExecType / StopTrail: 0 / 5, Position Size: 1
2020-01-05 14:30:00 TRAIL SELL Completed : Price: 7401.23, Cost: 7459.91, Comm: 11.10, ExecType / StopTrail: 5 / 5, Position Size: 0
2020-01-05 14:30:00 	TRADE: Trade closed. Gross PnL: -58.68, Net PnL: -80.97
2020-01-06 07:30:00 		BUY 7551.44, Cash : 99962.48, Close : 7551.44
2020-01-06 08:30:00 BUY Completed : Price: 7551.47, Cost: 7551.47, Comm: 11.33, ExecType / StopTrail: 0 / 5, Position Size: 1

In [206]:

# Get analyzers
sharpe_ratio_analysis = results[0].analyzers.sharpe_ratio.get_analysis()
trade_analyzer_analysis = results[0].analyzers.trade_analyzer.get_analysis()


In [207]:

# Get the total number of trades
# total_trades = trade_analyzer.total.trades
total_trades = trade_analyzer_analysis.get('total',0)
print(f"Total trades: {total_trades}")

# Print the starting cash
print('End Value: %.2f' % cerebroz.broker.getvalue())

fval = cerebroz.broker.getvalue()
returns = 100*( fval - cashinit ) / ( cashinit )
print(f"Returns: {returns:.2f}%")

# Drawdown analysis
drawdown_analysis = results[0].analyzers.drawdown.get_analysis()
print('Max Drawdown:', str( np.round ( drawdown_analysis.max.drawdown ,2 )  ) + '%' )


Total trades: AutoOrderedDict([('total', 122), ('open', 1), ('closed', 121)])
End Value: 65139.98
Returns: -34.86%
Max Drawdown: 61.49%


In [196]:

# Access the strategy and convert to DataFrame
strategy = cerebroz.runstrats[0][0]

bb_data = strategy.bb_data
print(type(bb_data))
print(bb_data.keys())

for bb_key in bb_data.keys():
  print(bb_key, len(bb_data[bb_key]))

bb_df = pd.DataFrame(strategy.bb_data)

print(bb_df.cross_up.sum()) #1891
print(bb_df.cross_dwn.sum()) #1866

print(bb_df.buyz.sum()) #779
print(bb_df.sellz.sum()) #778

del strategy, bb_data


<class 'dict'>
dict_keys(['date', 'bbtop', 'cclose', 'high_since_entry', 'trail_stop_price', 'cross_up', 'cross_dwn', 'cash', 'buyz', 'sellz', 'selltype', 'pos_size', 'pnl', 'commission'])
date 34991
bbtop 34991
cclose 34991
high_since_entry 34991
trail_stop_price 34991
cross_up 34991
cross_dwn 34991
cash 34991
buyz 34991
sellz 34991
selltype 34991
pos_size 34991
pnl 34991
commission 34991
1296.0
1293.0
552
552


In [177]:

display(bb_df[0:100])


,date,bbtop,cclose,high_since_entry,trail_stop_price,cross_up,cross_dwn,cash,buyz,sellz,selltype,pos_size,pnl,commission
0,2020-01-01 17:30:00,7244.028710,7225.59,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
1,2020-01-01 18:30:00,7244.657923,7221.20,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
2,2020-01-01 19:30:00,7238.158238,7221.43,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
3,2020-01-01 20:30:00,7240.813997,7234.19,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
4,2020-01-01 21:30:00,7248.303414,7245.37,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
5,2020-01-01 22:30:00,7251.104412,7236.83,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
6,2020-01-01 23:30:00,7254.539149,7240.67,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
7,2020-01-02 00:30:00,7256.211833,7229.48,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
8,2020-01-02 01:30:00,7258.166131,7236.73,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0
9,2020-01-02 02:30:00,7255.220905,7233.17,NaN,NaN,0.0,0.0,100000.00,False,False,-,0,0.0,0.0


In [48]:

bb_df.to_csv(f"{output_path}/bb_df_20240530.csv", index = False)


In [49]:

# Print the Sharpe Ratio
print('Sharpe Ratio:', sharpe_ratio_analysis['sharperatio'])

# Print the Drawdown
print('\nDrawdown:')
print('Max Drawdown:', drawdown_analysis.max.drawdown)
print('Max Drawdown Period:', drawdown_analysis.max.len)


Sharpe Ratio: -0.8124473712942424

Drawdown:
Max Drawdown: 56.17423789798526
Max Drawdown Period: 25037


In [ ]:

# Using custom analyzer to get he final trades
## ERROR: price isn't accurate when selling

final_trades_raw = results[0].analyzers.final_trades.get_analysis()
final_trades = pd.DataFrame(final_trades_raw)

del final_trades_raw

print(final_trades.columns.values.tolist())
print(final_trades.shape)

display(final_trades.tail(5))


AttributeError: 'ItemCollection' object has no attribute 'final_trades'

In [ ]:

print( 'Sharpe Ratio:', sharpe_ratio_analysis['sharperatio'] )


In [ ]:

bb_df.head(20)


In [ ]:

bb_df.to_csv(f"{output_path}/bb_df.csv", index = False)


In [ ]:

print(pricesdf.columns.values.tolist())
print(pricesdf.shape)


In [ ]:

%%time

del cerebro
del BollingerBandsStrategy

import backtrader as bt
import pandas as pd

class BollingerBandsStrategy(bt.Strategy):
    params = ( ('period', 15), ('devfactor', 1.5), )

    def __init__(self):
        self.dataclose = self.data.close
        self.boll = bt.indicators.BollingerBands(self.dataclose, period=self.params.period, devfactor=self.params.devfactor)

        # Reverse Logic
        self.buy_signal = bt.indicators.CrossUp(self.dataclose, self.boll.lines.top)
        self.sell_signal = bt.indicators.CrossDown(self.dataclose, self.boll.lines.bot)

    def next(self):
        if not self.position:  # Not in the market
            if self.buy_signal > 0:  # Crosses above the upper band
                self.buy(size=1)
        else:
            if self.sell_signal > 0:  # Crosses below the lower band
                self.sell(size=self.position.size)  # Close long position

# Load your DataFrame
# pricesdf = pd.read_csv('path_to_your_csv.csv', index_col='datetime', parse_dates=True)

# Ensure the DataFrame has the correct columns
assert all(col in pricesdf.columns for col in ['open', 'high', 'low', 'close', 'volume']), "DataFrame must contain 'open', 'high', 'low', 'close', and 'volume' columns"

# Create a data feed
data = bt.feeds.PandasData(dataname=pricesdf)

# Create a Cerebro engine instance
cerebro = bt.Cerebro()

# Add the strategy to Cerebro
cerebro.addstrategy(BollingerBandsStrategy)

# Add the data to Cerebro
cerebro.adddata(data)

# Set the initial cash
cerebro.broker.set_cash(10000)

# Set the commission
cerebro.broker.setcommission(commission=0.0015)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Print the starting cash
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

# Print the final cash
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Get analyzers
sharpe_ratio = results[0].analyzers.sharpe_ratio.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()
trade_analyzer = results[0].analyzers.trade_analyzer.get_analysis()

# Print the Sharpe Ratio
print('\nSharpe Ratio:')
print('Sharpe Ratio:', sharpe_ratio['sharperatio'])

# Print the Drawdown
print('\nDrawdown:')
print('Max Drawdown:', drawdown.max.drawdown)
print('Max Drawdown Period:', drawdown.max.len)


In [ ]:

# Print the Trade Analysis

print('\nTrade Analysis:')

print('Total Trades:', trade_analyzer.total.total)
print('Total Open:', trade_analyzer.total.open)
print('Total Closed:', trade_analyzer.total.get('closed', 0))
print('Winning Trades:', trade_analyzer.won.total)
print('Losing Trades:', trade_analyzer.lost.total)

# Plot the result
# cerebro.plot()


In [ ]:

%%time
# del cerebro
# del BBandsCross

import backtrader as bt
import pandas as pd

# Define a strategy
class BBandsCross(bt.Strategy):
    params = (
        ('bb_period', 20),
        ('bb_dev', 2.0),
        ('stake', 1),
    )

    def __init__(self):
        self.bbands = bt.indicators.BollingerBands(period=self.params.bb_period, devfactor=self.params.bb_dev)
        self.cross_up = bt.indicators.CrossUp(self.data.close, self.bbands.lines.top)
        self.cross_down = bt.indicators.CrossDown(self.data.close, self.bbands.lines.bot)

    def next(self):
        if not self.position:
            if self.cross_up > 0:
                self.buy(size=self.params.stake)
        else:
            if self.cross_down > 0:
                self.sell(size=self.params.stake)

# Create a DataFrame with your close prices
# Assuming 'close_prices' is your DataFrame with the 'Date' column as index and 'Close' column with prices
data = bt.feeds.PandasData(dataname=pricesdf)

# Create a Cerebro engine instance
cerebro = bt.Cerebro()

# Add data to Cerebro
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(BBandsCross)

# Set broker commission
cerebro.broker.setcommission(commission=0.0015)

# Set the desired cash start
cerebro.broker.set_cash(100000)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Run the backtest
results = cerebro.run()

# Plot the results
cerebro.plot()


In [ ]:

del cerebro
del BBandsCross

import backtrader as bt
import pandas as pd

# Define a strategy
class BBandsCross(bt.Strategy):
    params = (
        ('bb_period', 15),
        ('bb_dev', 1.5),
        ('stake', 1),
    )

    def __init__(self):
        self.bbands = bt.indicators.BollingerBands(period=self.params.bb_period, devfactor=self.params.bb_dev)
        self.cross_up = bt.indicators.CrossUp(self.data.close, self.bbands.lines.top)
        self.cross_down = bt.indicators.CrossDown(self.data.close, self.bbands.lines.bot)

    def next(self):
        if not self.position:
            if self.cross_up > 0:
                self.buy(size=self.params.stake)
        else:
            if self.cross_down > 0:
                self.sell(size=self.params.stake)

# Create a DataFrame with your close prices
# Assuming 'close_prices' is your DataFrame with the 'Date' column as index and 'Close' column with prices
data = bt.feeds.PandasData(dataname=pricesdf)

# Create a Cerebro engine instance
cerebro = bt.Cerebro()

# Add data to Cerebro
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(BBandsCross)

# Set broker commission
cerebro.broker.setcommission(commission=0.0015)

# Set the desired cash start
cerebro.broker.set_cash(100000)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

# Run the backtest
results = cerebro.run()
strategy = results[0]

# Print the Sharpe ratio
sharpe_ratio = strategy.analyzers.sharpe.get_analysis()
print(f"Sharpe Ratio: {sharpe_ratio['sharperatio']}")

# Print the trade analysis
trade_analysis = strategy.analyzers.trades.get_analysis()
print(f"Total Trades: {trade_analysis.total.total}")
print(f"Total Profit: {trade_analysis.pnl.net.total}")
print(f"Total Won: {trade_analysis.won.total}")
print(f"Total Lost: {trade_analysis.lost.total}")
print(f"Win Rate: {trade_analysis.won.total / trade_analysis.total.total * 100:.2f}%")

# Print returns
returns = strategy.analyzers.returns.get_analysis()
print(f"Total Returns: {returns['rtot'] * 100:.2f}%")
print(f"Annualized Returns: {returns['rnorm'] * 100:.2f}%")

# Plot the results
cerebro.plot()



In [ ]:

%%time

del cerebro
del BollingerBandsStrategy

import backtrader as bt
import pandas as pd

class BollingerBandsStrategy(bt.Strategy):
    params = (
        ('period', 15),
        ('devfactor', 1.5),
        ('trail_perc', 0.01),  # 1% trailing stop loss
    )

    def __init__(self):
        self.dataclose = self.data.close
        self.boll = bt.indicators.BollingerBands(self.dataclose, period=self.params.period, devfactor=self.params.devfactor)

        # self.buy_signal = bt.indicators.CrossOver(self.dataclose, self.boll.lines.bot)
        # self.sell_signal = bt.indicators.CrossOver(self.dataclose, self.boll.lines.top)

        # CORRECT WAY ACCORDING TO THE VIDEOS ON YOUTUBE
        # self.buy_signal = bt.indicators.CrossDown(self.dataclose, self.boll.lines.bot)
        # self.sell_signal = bt.indicators.CrossUp(self.dataclose, self.boll.lines.top)

        self.buy_signal = bt.indicators.CrossUp(self.dataclose, self.boll.lines.top)
        self.sell_signal = bt.indicators.CrossDown(self.dataclose, self.boll.lines.bot)

        self.highest_price = None
        self.lowest_price = None

    def next(self):
        if not self.position:  # Not in the market
            if self.buy_signal > 0:  # Crosses above the lower band
                self.buy(size=1)
                self.highest_price = self.dataclose[0]
            elif self.sell_signal < 0:  # Crosses below the upper band
                self.sell(size=1)
                self.lowest_price = self.dataclose[0]
        else:
            if self.position.size > 0:  # Long position
                if self.dataclose[0] > self.highest_price:
                    self.highest_price = self.dataclose[0]

                stop_price = self.highest_price * (1 - self.params.trail_perc)

                if self.dataclose[0] <= stop_price:
                    self.sell(size=self.position.size)  # Close long position
                elif self.sell_signal > 0:  # Crosses below the upper band
                    self.sell(size=self.position.size)  # Close long position
                    self.sell(size=1)  # Enter short position
                    self.lowest_price = self.dataclose[0]

            elif self.position.size < 0:  # Short position
                if self.dataclose[0] < self.lowest_price:
                    self.lowest_price = self.dataclose[0]

                stop_price = self.lowest_price * (1 + self.params.trail_perc)

                if self.dataclose[0] >= stop_price:
                    self.buy(size=-self.position.size)  # Cover short position
                elif self.buy_signal < 0:  # Crosses above the lower band
                    self.buy(size=-self.position.size)  # Cover short position
                    self.buy(size=1)  # Enter long position
                    self.highest_price = self.dataclose[0]


# Load your DataFrame
# pricesdf = pd.read_csv('path_to_your_csv.csv', index_col='datetime', parse_dates=True)

# Create a data feed
data = bt.feeds.PandasData(dataname=pricesdf)

# Create a Cerebro engine instance
cerebro = bt.Cerebro()

# Add the strategy to Cerebro
cerebro.addstrategy(BollingerBandsStrategy)

# Add the data to Cerebro
cerebro.adddata(data)

# Set the initial cash
cerebro.broker.set_cash(10000)

# Set the commission
cerebro.broker.setcommission(commission=0.0015)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Print the starting cash
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

# Print the final cash
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Get analyzers
sharpe_ratio = results[0].analyzers.sharpe_ratio.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()
trade_analyzer = results[0].analyzers.trade_analyzer.get_analysis()

# Print the Sharpe Ratio
print('\nSharpe Ratio:')
print('Sharpe Ratio:', sharpe_ratio['sharperatio'])

# Print the Drawdown
print('\nDrawdown:')
print('Max Drawdown:', drawdown.max.drawdown)
print('Max Drawdown Period:', drawdown.max.len)

# Print the Trade Analysis
print('\nTrade Analysis:')
print('Total Trades:', trade_analyzer.total.total)
print('Total Open:', trade_analyzer.total.open)
print('Total Closed:', trade_analyzer.total.closed)
print('Winning Trades:', trade_analyzer.won.total)
print('Losing Trades:', trade_analyzer.lost.total)

# Plot the result
cerebro.plot()


In [ ]:


# Get analyzers
sharpe_ratio = results[0].analyzers.sharpe_ratio.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()
trade_analyzer = results[0].analyzers.trade_analyzer.get_analysis()

# Print the Sharpe Ratio
print('\nSharpe Ratio:')
print('Sharpe Ratio:', sharpe_ratio['sharperatio'])

# Print the Drawdown
print('\nDrawdown:')
print('Max Drawdown:', drawdown.max.drawdown)
print('Max Drawdown Period:', drawdown.max.len)

# Print the Trade Analysis
print('\nTrade Analysis:')
print('Total Trades:', trade_analyzer.total.total)
print('Total Open:', trade_analyzer.total.open)
print('Total Closed:', trade_analyzer.total.closed)
print('Winning Trades:', trade_analyzer.won.total)
print('Losing Trades:', trade_analyzer.lost.total)

# Plot the result
cerebro.plot()


In [ ]:

"""
data = bt.feeds.YahooFinanceCSVData(
    dataname='path/to/data.csv',
    fromdate=datetime(2020, 1, 1),
    todate=datetime(2021, 1, 1),
)
"""

data = bt.feeds.PandasData(dataname=pricesdf)
# print(type(data))

cerebro.adddata(data)


In [ ]:

%%time

# Set initial cash and sizer
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0, annualized=True)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)

# Run the backtest
results = cerebro.run()


In [ ]:
results

In [ ]:


# Print the analyzer results
strat = results[0]
sharperatio = strat.analyzers.sharperatio.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()

print(f"Sharpe Ratio: {sharperatio['sharperatio']:.2f}")
print(f"Max Drawdown: {drawdown['max']['moneydown']:.2f}")
print(f"Total Trades: {trade_analysis['total']['total']}")
print(f"Profitable Trades: {trade_analysis['won']['total']}")
print(f"Loss Trades: {trade_analysis['lost']['total']}")
print(f"Average Profit: {trade_analysis['won']['pnl']:.2f}")
print(f"Average Loss: {trade_analysis['lost']['pnl']:.2f}")

In [ ]:

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

cerebro.run()
cerebro.plot()


In [ ]:
EXPORT = True

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print(statsdf.columns.values.tolist())

# Assuming df is your DataFrame with the data
# Pivot the DataFrame to format suitable for heatmap
# pivot_table = statsdfexp.pivot("st_period", "st_multiplier", "Total Return [%]")
pivot_table = statsdf.pivot(index='bbands-basic_bbwin', columns='bbands-basic_alp', values='Total Return [%]')

# Plotting the heatmap
plt.figure(figsize=(10, 8))
# sns.heatmap(pivot_table, annot=False, fmt=".1f", cmap="coolwarm")
ax = sns.heatmap(pivot_table, annot=False, fmt=".1f", cmap="magma", linewidths=.5) #cmap = 'coolwarm'
plt.title('Heatmap of Total Return [%] by st_period and st_mul')
# plt.show()

# Inverting the y-axis
ax.invert_yaxis()

chartnm = f"{symbol}_stats_{ctime}_macross-{bbtypef}-optim-heatmap"
filepath = f"/content/drive/MyDrive/Quantvaala/logs/{chartnm}" + ".png"  # You can change the directory path and file extension

if EXPORT:
  # Save the figure
  plt.savefig(filepath)



In [ ]:

to_plot = 'Total Trades'
trcutoff = 550

# Apply filters to the DataFrame
# Example filters: (you can change these according to your needs)
# filtered_df = statsdf[(statsdf['Total Return [%]'] > 500) & (statsdfexp['Total Trades'] >= 104)]
filtered_df = statsdf[(statsdf['Total Return [%]'] > trcutoff)]

# Pivot the DataFrame to format suitable for heatmap
pivot_table = filtered_df.pivot(index='bbands-basic_bbwin', columns='bbands-basic_alp', values= to_plot)

# Plotting the heatmap
plt.figure(figsize=(10, 8))
ax = sns.heatmap(pivot_table, annot=False, fmt=".1f", cmap="magma", linewidths=.5)
plt.title(f'Heatmap of "{to_plot}" by bbands-basic_bbwin and bbands-basic_alp, with Total Return % cut-off {trcutoff}')

# Inverting the y-axis
ax.invert_yaxis()

chartnm = f"{symbol}_stats_{ctime}_bbands-{bbtypef}-optim-heatmap-filt"
filepath = f"/content/drive/MyDrive/Quantvaala/logs/{chartnm}" + ".png"  # You can change the directory path and file extension

if EXPORT:
  # Save the figure
  plt.savefig(filepath)

# Display the heatmap
plt.show()


In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

# Ensure your DataFrame 'statsdfexp' is defined and has the correct structure
to_plot = 'Total Trades'
trcutoff = 550
slider_col = 'Max Drawdown [%]'

# Function to update the plot based on the slider
def update_plot(drawdown_cutoff):
    # Apply filters to the DataFrame including the slider for 'Max Drawdown [%]'
    filtered_df = statsdfexp[(statsdfexp['Total Return [%]'] > trcutoff) & (statsdfexp[slider_col] <= drawdown_cutoff)]

    # Pivot the DataFrame to format suitable for heatmap
    pivot_table = filtered_df.pivot(index='bbands-basic_bbwin', columns='bbands-basic_alp', values=to_plot)

    # Plotting the heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(pivot_table, annot=False, fmt=".1f", cmap="magma", linewidths=.5)
    plt.title(f'Heatmap of "{to_plot}" by bbands-basic_bbwin and bbands-basic_alp, with Total Return % cut-off {trcutoff}')

    # Inverting the y-axis
    ax.invert_yaxis()

    plt.show()

# Slider widget for 'Max Drawdown [%]'
drawdown_slider = IntSlider(
    min=statsdf[slider_col].min(),
    max=statsdf[slider_col].max(),
    step=1,  # Adjust step if needed for more fine-grained control
    value=statsdf[slider_col].median(),  # Setting initial value to median for a sensible default
    description='Drawdown Cutoff'
)

# Display the widget and connect to the update function
interact(update_plot, drawdown_cutoff=drawdown_slider)


In [ ]:
statsdf.sample(5)

In [ ]:

# Define bins and labels
bins = list(range(-1000, 0, 1000)) + list(range(0, 501, 250)) + list(range(550, 1501, 250)) + [float('inf')]
labels = [f"{bins[i]} to {bins[i+1]-1}" for i in range(len(bins)-1)]

print(labels)
# Bucketing
statsdf['rr'] = pd.cut(statsdf['Total Return [%]'], bins=bins, labels=labels, right=False)



In [ ]:


# Group by 'Return Range' and aggregate the data
aggregated_data = statsdf.groupby('rr').agg({
    'Win Rate [%]': ['min', 'median', 'max'],
    'Total Trades': ['min', 'median', 'max'],
    'Sharpe Ratio': ['min', 'median', 'max'],
    'Total Return [%]': 'count'
})

# Fill NaN values in specific columns before conversion
aggregated_data['Win Rate [%]'] = aggregated_data['Win Rate [%]'].fillna(0)  # or another placeholder value if more appropriate
aggregated_data['Total Trades'] = aggregated_data['Total Trades'].fillna(0)

# Apply integer conversion selectively
int_columns = ['Win Rate [%]', 'Total Trades']
for col in int_columns:
    aggregated_data[col] = aggregated_data[col].applymap(int)

# Round 'Sharpe Ratio' to two decimals
aggregated_data['Sharpe Ratio'] = aggregated_data['Sharpe Ratio'].applymap(lambda x: round(x, 2))

display(aggregated_data)


In [ ]:
EXPORT = True

In [ ]:

if EXPORT:
  statsdfexp = statsdf.copy()

  statsdfexp['Start'] = statsdfexp['Start Index'].dt.tz_localize(None)
  statsdfexp['End'] = statsdfexp['End Index'].dt.tz_localize(None)

  filenm = f"{symbol}_stats_{ctime}_bbands-{bbtypef}-optim.xlsx"
  print(filenm)

  statsdfexp.to_excel(f"/content/drive/MyDrive/Quantvaala/logs/{filenm}")

  print()
  print(datetime.datetime.now(local_tz))

  del statsdfexp
